In [432]:
import re
import json
import time
import pickle
from collections import defaultdict

import spacy
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [438]:
%%time
OPTIONS = Options()
OPTIONS.add_argument('--headless')
# make sure Homebrew is installed
# then run 'brew tap homebrew/cask && brew cask install chromedriver'
driver = webdriver.Chrome(options=OPTIONS)

CPU times: user 4.42 ms, sys: 9.63 ms, total: 14 ms
Wall time: 1.26 s


In [70]:
# python3 -m spacy download en
nlp = spacy.load('en')

def tokenize(line):
    return [(token.text, token.tag_) for token in nlp(line)]

In [71]:
def numerical(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not numerical
        if not pair[1] == "CD":
            return False
    return True

def nouns_only(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not a noun or cardinal number
        if not (pair[1] == "NN" or pair[1] == "NNS"):
            return False
    return True

In [477]:
any(numerical(i) for i in '9 cups'.split())

True

In [72]:
type_exceptions = ['can', 'tablespoon']

for exception in type_exceptions:
    print(tokenize(exception), spacy.explain(tokenize(exception)[0][1]))

[('can', 'MD')] verb, modal auxiliary
[('tablespoon', 'VB')] verb, base form


In [535]:
def extract_quantity_in_backets(line):
    # find '(abc)' where 'abc' is in arbitrary length and 'abc' does not contain brackets
    pattern = re.compile(r'\([^\(\)]*\)') 
    match = re.findall(pattern, line)
    if len(match) != 0:
        # if no numerical value or line_split length > 3 
        if not any(char.isdigit() for char in match[0]) or len(match[0].split()) > 3:
            return None
        return match
    
def extract_preparation(line):
    # find ', abc' where 'abc' is in arbitrary length
    pattern = re.compile(r'[\b]?, [^\(\)]*')
    match = re.findall(pattern, line)
    if len(match) != 0:
        return match
    
def extract_all(line):
#     print(line)
    type_exceptions = ['can', 'tablespoon', 'oz']
    quantity_split = []
    measurement = None
    
    # extract preparation
    line = line.replace(' -', ',')
    preparation = extract_preparation(line)
    if preparation:
#         print('preparation[0]: {0}|'.format(preparation[0]))
        preparation = preparation[0].strip()
        line = re.sub(r'{0}'.format(preparation), '', line)
        # remove ', ' prefix
        preparation = preparation[2:]
        
    # extract quantity in backets
    quantity_in_brackets = extract_quantity_in_backets(line)
    if quantity_in_brackets:
        line = re.sub(r'\({0}\)'.format(quantity_in_brackets[0]), '', line)
        quantity_in_brackets = quantity_in_brackets[0]
    
    line_split = line.split()
    # extract quantity from the first word if the word contains a digit
    if any(char.isdigit() for char in line_split[0]):
        quantity_split.append(line_split[0])
    
        # extract quantity from the second word if the word contains a digit
        if any(char.isdigit() for char in line_split[1]):
            quantity_split.append(line_split[1])
            # check measurement type
            # to avoid case like '1 large tomato, seeded and chopped'
            if nouns_only(line_split[2]) or line_split[2] in type_exceptions:
                measurement = line_split[2]
        else:
            # check line_split length for case like '1 egg' or '1/2 onion, chopped'
            if len(line_split) > 2 and (nouns_only(line_split[1]) or line_split[1] in type_exceptions):
                measurement = line_split[1]
        line = re.sub(r'{0}'.format(' '.join(quantity_split)), '', line)
    
    if measurement:
        line = re.sub(r'{0}'.format(measurement), '', line)
    
    # append quantity in backets at the end
    if quantity_in_brackets:
        quantity_split.append(quantity_in_brackets)
    quantity = ' '.join(quantity_split)

    # extract ingredient name
    line = re.sub(r'[ ]?®', '', line)
    line = re.sub(r'[ ]?™', '', line)
    ingredient_name = line.strip()
    
    # if 'or to taske' or 'or as needed' in preparation
    if preparation is not None and 'or ' in preparation:
        quantity += ' ' + preparation
        preparation = None    
    
    return quantity, measurement, ingredient_name, preparation

In [536]:
# test on tricky cases
# extract_all('4 roma (plum) tomatoes, chopped')
# extract_all('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')
# extract_all('1 clove garlic, minced, or more to taste')
# extract_all('4 pounds Korean-style short ribs (beef chuck flanken, cut 1/3 to 1/2 inch thick across bones)')
# extract_all('1/2 (14 ounce) package vegetarian ground beef (e.g., Gimme Lean TM)')
# extract_all('1 0.42 oz packet concentrated vegetable base (such as Swanson® Vegetable Flavor Boost®) (optional)')
# extract_all('2 avocados - peeled, pitted, and sliced')
# extract_all('1 1/2 (8 ounce) packages seitan')
extract_all('1 (15 ounce) can garbanzo beans (chickpeas), drained and rinsed')

('1 (15 ounce)', 'can', 'garbanzo beans (chickpeas)', 'drained and rinsed')

In [537]:
%%time
# test
get_ingredient_list('https://www.allrecipes.com/recipe/13928/enchilada-casserole/')

CPU times: user 98.5 ms, sys: 1.24 ms, total: 99.8 ms
Wall time: 4.05 s


{'1 (15 ounce) can black beans, rinsed and drained',
 '1 (19 ounce) can enchilada sauce',
 '1 (4 ounce) can diced green chile peppers',
 '1 (6 ounce) can sliced black olives',
 '1 (8 ounce) package tempeh, crumbled',
 '1 jalapeno pepper, seeded and minced',
 '1 onion, chopped',
 '2 cloves garlic, minced',
 '6 (6 inch) corn tortillas',
 '8 ounces shredded Cheddar cheese'}

In [483]:
def extract_descriptor(ingredient_name):
    type_exceptions = ['parsley', 'garlic', 'chili']
    descriptor = []
    token_tag_pairs = []
    
    for element in ingredient_name.split():
        # treat compound word with hyphen as an adjective
        if '-' in element:
            token_tag_pairs.append((element, 'JJ'))
        else:
            token_tag_pairs.append([(token.text, token.tag_) for token in nlp(element)][0])
    
    for pair in token_tag_pairs:
        # if the word is an adjective, an adverb, or a past participle of a verb, or exception like 'ground'
        if pair[1] == "JJ" or pair[1] == "RB" or pair[1] == "VBN" or pair[0] == 'ground':
            if pair[0] not in type_exceptions:
                descriptor.append(pair[0])
    if len(descriptor) != 0:
        return ' '.join(descriptor)

In [484]:
# test
# extract_descriptor('extra-firm tofu') 
# extract_descriptor('all-purpose flour') 
# extract_descriptor('freshly ground black pepper') 
extract_descriptor('1/2 cup chopped parsley')
extract_descriptor('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')

'colossal EZ-peel split back'

In [485]:
def extract_ingredients_nouns(line):
    ingredients_nouns = set()
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        if pair[1] == 'NN' or pair[1] == 'NNS':
            ingredients_nouns |= {pair[0]}
    return ingredients_nouns

In [553]:
def get_ingredient_list(url):
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
    global driver
    try:
        driver.get(url)
    except:
        print('relaunching webdriver...')
        driver = webdriver.Chrome(options=OPTIONS)
        driver.get(url)
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # extract ingredients section from the webpage
    ingredients = set([element.label.text.strip() for element in soup.find_all(class_='checkList__line')])
    # remove unnecessary elements
    unnecessary = ['', 'Add all ingredients to list']
    for i in unnecessary:
        if i in ingredients:
            ingredients.remove(i)
    return ingredients

In [488]:
def lemmatize(line):
    if len(line.split()) == 1:
        return [token.lemma_ for token in nlp(line)][0]
    return line

In [489]:
def get_ingredients_from_url(url):
    print(url)
    ingredients_url = set()
    ingredients_measurement_dict_url = defaultdict(set)
    ingredients_descriptor_dict_url = defaultdict(set)
    ingredients_preparation_dict_url = defaultdict(set)
    
    ingredient_list = get_ingredient_list(url)
    for line in ingredient_list:

        
        quantity, measurement, ingredient_name, preparation = extract_all(line)
        # exceptions like "topping:"
        if ':' in line:
            print(line)
            print(url)
            continue
            
        descriptor = extract_descriptor(ingredient_name)
        # remove descriptor if not None
        ingredient = ingredient_name.replace(descriptor, '').strip() if descriptor else ingredient_name
        # if ingredient is empty after removing descriptor
        if ingredient == '':
            ingredient = ingredient_name
        # remove ' to taste' in ingredient if any
        ingredient = re.sub(r'(or)? to taste', '', ingredient)
        ingredient = ' '.join(ingredient.split())
        # lemmatize ingredient
#         ingredient = lemmatize(ingredient)
        
        if ingredient not in ingredients_measurement_dict_url:
            ingredients_measurement_dict_url[ingredient] |= {measurement}
        if ingredient not in ingredients_descriptor_dict_url:
            ingredients_descriptor_dict_url[ingredient] |= {descriptor}
        if ingredient not in ingredients_preparation_dict_url:
            ingredients_preparation_dict_url[ingredient] |= {preparation}
        ingredients_url |= {ingredient}
        
        # test
        print(line)
        print('quantity:', quantity)
        print('measurement:', measurement)
        print('ingredient name:', ingredient_name)
        print('descriptor:', descriptor)
        print('ingredient without descriptor:', ingredient)
        print('preparation:', preparation)
        print()
    
    return ingredients_url, ingredients_measurement_dict_url, ingredients_descriptor_dict_url, ingredients_preparation_dict_url

In [490]:
def get_recipe_urls(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    recipe_cards = soup.find_all('div', class_='fixed-recipe-card__info')
    recipe_urls = []
    for recipe_card in recipe_cards:
        url = recipe_card.find('a')['href']
        recipe_urls.append(url)
    return recipe_urls

In [492]:
def get_all_ingredients_in_category(category_url):
    ingredients_freq_dict_all = dict()
    ingredients_measurement_dict_all = defaultdict(set)
    ingredients_descriptor_dict_all = defaultdict(set)
    ingredients_preparation_dict_all = defaultdict(set)
    
    recipe_urls = get_recipe_urls(category_url)
    for recipe_url in recipe_urls:
        recipe_ingredients, recipe_ingredients_measurement_dict, ingredients_descriptor_dict, ingredients_preparation_dict = get_ingredients_from_url(recipe_url)
        for i in recipe_ingredients:
            if i in ingredients_freq_dict_all:
                ingredients_freq_dict_all[i] += 1
            else:
                ingredients_freq_dict_all[i] = 1
                
        for i in recipe_ingredients:
            if i in ingredients_measurement_dict_all:
                ingredients_measurement_dict_all[i] |= recipe_ingredients_measurement_dict[i]
            else:
                ingredients_measurement_dict_all[i] = recipe_ingredients_measurement_dict[i]
    
        for i in recipe_ingredients:
            if i in ingredients_descriptor_dict_all:
                ingredients_descriptor_dict_all[i] |= ingredients_descriptor_dict[i]
            else:
                ingredients_descriptor_dict_all[i] = ingredients_descriptor_dict[i]
                
        for i in recipe_ingredients:
            if i in ingredients_preparation_dict_all:
                ingredients_preparation_dict_all[i] |= ingredients_preparation_dict[i]
            else:
                ingredients_preparation_dict_all[i] = ingredients_preparation_dict[i]  
                
        # scraping pause to avoid being banned by site
#         time.sleep(1)
    return ingredients_freq_dict_all, ingredients_measurement_dict_all, ingredients_descriptor_dict_all, ingredients_preparation_dict_all

In [493]:
# 4 pages of vegetarian protein recipes
# 6 pages of meat recipes
# 2 pages of seafood recipes

In [538]:
%%time
vegetarian_protein_ingredients_freq_dict = dict()
vegetarian_protein_ingredients_measurement_dict = defaultdict(set)
vegetarian_protein_ingredients_descriptor_dict = defaultdict(set)
vegetarian_protein_ingredients_preparation_dict = defaultdict(set)
    
for page in range(4):
    vegetarian_protein_url = 'https://www.allrecipes.com/recipes/16778/everyday-cooking/vegetarian/protein/?page=' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(vegetarian_protein_url)
    for i in ingredients_cat.keys():
        if i in vegetarian_protein_ingredients_freq_dict:
            vegetarian_protein_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            vegetarian_protein_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in vegetarian_protein_ingredients_measurement_dict:
            vegetarian_protein_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            vegetarian_protein_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in vegetarian_protein_ingredients_descriptor_dict:
            vegetarian_protein_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            vegetarian_protein_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in vegetarian_protein_ingredients_preparation_dict:
            vegetarian_protein_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            vegetarian_protein_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/88207/tofu-and-rice-stuffed-peppers/
2 cups water
quantity: 2
measurement: cups
ingredient name: water
descriptor: None
ingredient without descriptor: water
preparation: None

1 (12 ounce) package extra-firm tofu, drained and diced
quantity: 1 (12 ounce)
measurement: package
ingredient name: extra-firm tofu
descriptor: extra-firm
ingredient without descriptor: tofu
preparation: drained and diced

1 3/4 cups marinara sauce, divided
quantity: 1 3/4
measurement: cups
ingredient name: marinara sauce
descriptor: None
ingredient without descriptor: marinara sauce
preparation: divided

2 red bell peppers, halved and seeded
quantity: 2
measurement: None
ingredient name: red bell peppers
descriptor: red
ingredient without descriptor: bell peppers
preparation: halved and seeded

salt to taste
quantity: 
measurement: None
ingredient name: salt to taste
descriptor: None
ingredient without descriptor: salt
preparation: None

2 orange bell peppers, halved and seeded

1 large tomato, seeded and chopped
quantity: 1
measurement: None
ingredient name: large tomato
descriptor: large
ingredient without descriptor: tomato
preparation: seeded and chopped

2 tablespoons vegetable oil
quantity: 2
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1 (14 ounce) package firm tofu, drained
quantity: 1 (14 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: drained

1 teaspoon minced fresh ginger
quantity: 1
measurement: teaspoon
ingredient name: minced fresh ginger
descriptor: minced fresh
ingredient without descriptor: ginger
preparation: None

2 cloves garlic, minced
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: minced

1 tablespoon sesame seeds
quantity: 1
measurement: tablespoon
ingredient name: sesame seeds
descriptor: None
ingredien

2 tablespoons corn oil
quantity: 2
measurement: tablespoons
ingredient name: corn oil
descriptor: None
ingredient without descriptor: corn oil
preparation: None

https://www.allrecipes.com/recipe/13896/tofu-parmigiana/
5 tablespoons grated Parmesan cheese
quantity: 5
measurement: tablespoons
ingredient name: grated Parmesan cheese
descriptor: grated
ingredient without descriptor: Parmesan cheese
preparation: None

1 (8 ounce) can tomato sauce
quantity: 1 (8 ounce)
measurement: can
ingredient name: tomato sauce
descriptor: None
ingredient without descriptor: tomato sauce
preparation: None

2 teaspoons dried oregano, divided
quantity: 2
measurement: teaspoons
ingredient name: dried oregano
descriptor: dried
ingredient without descriptor: oregano
preparation: divided

salt to taste
quantity: 
measurement: None
ingredient name: salt to taste
descriptor: None
ingredient without descriptor: salt
preparation: None

1/2 cup seasoned bread crumbs
quantity: 1/2
measurement: cup
ingredient name: 

1 red onion, sliced
quantity: 1
measurement: None
ingredient name: red onion
descriptor: red
ingredient without descriptor: onion
preparation: sliced

3 tablespoons peanut oil
quantity: 3
measurement: tablespoons
ingredient name: peanut oil
descriptor: None
ingredient without descriptor: peanut oil
preparation: None

1 teaspoon cornstarch
quantity: 1
measurement: teaspoon
ingredient name: cornstarch
descriptor: None
ingredient without descriptor: cornstarch
preparation: None

1 tablespoon brown sugar
quantity: 1
measurement: tablespoon
ingredient name: brown sugar
descriptor: brown
ingredient without descriptor: sugar
preparation: None

1 green chile pepper, chopped
quantity: 1
measurement: None
ingredient name: green chile pepper
descriptor: green chile
ingredient without descriptor: pepper
preparation: chopped

3 tablespoons white vinegar
quantity: 3
measurement: tablespoons
ingredient name: white vinegar
descriptor: white
ingredient without descriptor: vinegar
preparation: None

1 t

1 bunch fresh spinach, stems removed
quantity: 1
measurement: bunch
ingredient name: fresh spinach
descriptor: fresh
ingredient without descriptor: spinach
preparation: stems removed

1 tablespoon curry paste
quantity: 1
measurement: tablespoon
ingredient name: curry paste
descriptor: None
ingredient without descriptor: curry paste
preparation: None

https://www.allrecipes.com/recipe/88768/breaded-fried-softly-spiced-tofu/
1/2 teaspoon freshly ground black pepper
quantity: 1/2
measurement: teaspoon
ingredient name: freshly ground black pepper
descriptor: freshly ground black
ingredient without descriptor: pepper
preparation: None

3 tablespoons vegetable oil
quantity: 3
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

2 cups vegetable broth
quantity: 2
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

1 (16 ounce) package extr

...

3 tablespoons white sugar
quantity: 3
measurement: tablespoons
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

1/4 teaspoon red pepper flakes
quantity: 1/4
measurement: teaspoon
ingredient name: red pepper flakes
descriptor: red
ingredient without descriptor: pepper flakes
preparation: None

1 (8 ounce) package baked tofu, cut into bite-size pieces
quantity: 1 (8 ounce)
measurement: package
ingredient name: baked tofu
descriptor: baked
ingredient without descriptor: tofu
preparation: cut into bite-size pieces

3 green onions, chopped
quantity: 3
measurement: None
ingredient name: green onions
descriptor: green
ingredient without descriptor: onions
preparation: chopped

2 tablespoons soy sauce
quantity: 2
measurement: tablespoons
ingredient name: soy sauce
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

1 cup hot water
quantity: 1
measurement: cup
ingredient name: hot water
descriptor: hot
ingredient wit

salt and pepper to taste
quantity: 
measurement: None
ingredient name: salt and pepper to taste
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1 (16 ounce) package extra firm tofu
quantity: 1 (16 ounce)
measurement: package
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: None

3 tablespoons olive oil
quantity: 3
measurement: tablespoons
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: None

1/4 cup chopped green onions
quantity: 1/4
measurement: cup
ingredient name: chopped green onions
descriptor: chopped green
ingredient without descriptor: onions
preparation: None

1/3 cup soy sauce
quantity: 1/3
measurement: cup
ingredient name: soy sauce
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

3 cloves garlic, minced
quantity: 3
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: ga

4 whole wheat hamburger buns, split
quantity: 4
measurement: None
ingredient name: whole wheat hamburger buns
descriptor: whole
ingredient without descriptor: wheat hamburger buns
preparation: split

1 cup bread crumbs
quantity: 1
measurement: cup
ingredient name: bread crumbs
descriptor: None
ingredient without descriptor: bread crumbs
preparation: None

1 teaspoon kelp powder
quantity: 1
measurement: teaspoon
ingredient name: kelp powder
descriptor: None
ingredient without descriptor: kelp powder
preparation: None

olive oil, as needed
quantity: 
measurement: None
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: as needed

1/4 teaspoon paprika
quantity: 1/4
measurement: teaspoon
ingredient name: paprika
descriptor: None
ingredient without descriptor: paprika
preparation: None

1/4 teaspoon onion powder or flakes
quantity: 1/4
measurement: teaspoon
ingredient name: onion powder or flakes
descriptor: None
ingredient without descriptor: o

6 tablespoons rice vinegar
quantity: 6
measurement: tablespoons
ingredient name: rice vinegar
descriptor: None
ingredient without descriptor: rice vinegar
preparation: None

8 green onions, halved lengthwise
quantity: 8
measurement: None
ingredient name: green onions
descriptor: green
ingredient without descriptor: onions
preparation: halved lengthwise

1/2 (16 ounce) package linguine pasta
quantity: 1/2 (16 ounce)
measurement: package
ingredient name: linguine pasta
descriptor: linguine
ingredient without descriptor: pasta
preparation: None

1 tablespoon sesame oil
quantity: 1
measurement: tablespoon
ingredient name: sesame oil
descriptor: None
ingredient without descriptor: sesame oil
preparation: None

2 cloves garlic, chopped
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: chopped

3 tablespoons honey
quantity: 3
measurement: tablespoons
ingredient name: honey
descriptor: None
ingredient without descriptor:

5 small eggplants
quantity: 5
measurement: None
ingredient name: small eggplants
descriptor: small
ingredient without descriptor: eggplants
preparation: None

https://www.allrecipes.com/recipe/25917/vegan-chunky-chili/
1/4 onion, chopped
quantity: 1/4
measurement: None
ingredient name: onion
descriptor: None
ingredient without descriptor: onion
preparation: chopped

1/2 cup chopped green bell pepper
quantity: 1/2
measurement: cup
ingredient name: chopped green bell pepper
descriptor: chopped green
ingredient without descriptor: bell pepper
preparation: None

3/4 cup extra firm tofu, drained, crumbled
quantity: 3/4
measurement: cup
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: drained, crumbled

1/2 cup dry white beans, soaked overnight
quantity: 1/2
measurement: cup
ingredient name: dry white beans
descriptor: dry white
ingredient without descriptor: beans
preparation: soaked overnight

6 cups chopped fresh tomatoes
quantity: 6

In [539]:
sorted(vegetarian_protein_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True)

[('tofu', 66),
 ('onion', 39),
 ('salt', 34),
 ('garlic', 32),
 ('pepper', 31),
 ('oil', 27),
 ('soy sauce', 22),
 ('water', 21),
 ('olive oil', 19),
 ('bell pepper', 17),
 ('mushrooms', 14),
 ('onions', 14),
 ('sugar', 12),
 ('cilantro', 11),
 ('sesame oil', 10),
 ('garlic powder', 10),
 ('cumin', 9),
 ('carrots', 9),
 ('tomato sauce', 9),
 ('tomatoes', 9),
 ('celery', 8),
 ('tempeh', 8),
 ('broth', 8),
 ('basil', 8),
 ('mozzarella cheese', 7),
 ('coconut milk', 7),
 ('salt and pepper', 7),
 ('ginger root', 7),
 ('rice', 6),
 ('clove garlic', 6),
 ('spinach', 6),
 ('oregano', 6),
 ('curry powder', 6),
 ('cayenne pepper', 6),
 ('chili powder', 6),
 ('beans', 6),
 ('butter', 6),
 ('tamari', 5),
 ('soy milk', 5),
 ('ginger', 5),
 ('barbecue sauce', 5),
 ('Cheddar cheese', 5),
 ('peppers', 5),
 ('lime juice', 5),
 ('vinegar', 5),
 ('cornstarch', 5),
 ('parsley', 5),
 ('honey', 5),
 ('vanilla extract', 5),
 ('bell peppers', 4),
 ('tomato', 4),
 ('zucchini', 4),
 ('coriander', 4),
 ('yeast'

In [540]:
vegetarian_protein_ingredients_preparation_dict

defaultdict(set,
            {'olive oil': {None, 'as needed'},
             'water': {None},
             'rice': {None},
             'pepper': {None, 'chopped', 'seeded and minced'},
             'bell peppers': {None, 'halved and seeded'},
             'tomato': {None, 'chopped', 'seeded and chopped'},
             'marinara sauce': {'divided'},
             'clove garlic': {None, 'minced'},
             'salt': {None},
             'mozzarella cheese': {None, 'divided'},
             'tofu': {None,
              'crumbled',
              'cubed',
              'cut into 1-inch cubes',
              'cut into 1/4-inch cubes',
              'cut into 3/4 inch cubes',
              'cut into bite-size pieces',
              'cut into bite-sized cubes',
              'diced',
              'drained',
              'drained and cubed',
              'drained and cut into cubes',
              'drained and cut into strips',
              'drained and diced',
              'drained and p

In [554]:
%%time
meat_ingredients_freq_dict = dict()
meat_ingredients_measurement_dict = defaultdict(set)
meat_ingredients_descriptor_dict = defaultdict(set)
meat_ingredients_preparation_dict = defaultdict(set)

for index in range(200, 206):
    meat_url = 'https://www.allrecipes.com/recipes/{}/meat-and-poultry/'.format(index)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(meat_url)
    for i in ingredients_cat.keys():
        if i in meat_ingredients_freq_dict:
            meat_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            meat_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in meat_ingredients_measurement_dict:
            meat_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            meat_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in meat_ingredients_descriptor_dict:
            meat_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            meat_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in meat_ingredients_preparation_dict:
            meat_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            meat_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/17991/stuffed-green-peppers-i/
relaunching webdriver...
salt and pepper to taste
quantity: 
measurement: None
ingredient name: salt and pepper to taste
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1 cup shredded Cheddar cheese
quantity: 1
measurement: cup
ingredient name: shredded Cheddar cheese
descriptor: shredded
ingredient without descriptor: Cheddar cheese
preparation: None

salt to taste
quantity: 
measurement: None
ingredient name: salt to taste
descriptor: None
ingredient without descriptor: salt
preparation: None

1 (14.5 ounce) can whole peeled tomatoes, chopped
quantity: 1 (14.5 ounce)
measurement: can
ingredient name: whole peeled tomatoes
descriptor: whole peeled
ingredient without descriptor: tomatoes
preparation: chopped

1/3 cup chopped onion
quantity: 1/3
measurement: cup
ingredient name: chopped onion
descriptor: chopped
ingredient without descriptor: onion
preparation: None

6 green bell peppers


1/2 cup reduced-sodium soy sauce
quantity: 1/2
measurement: cup
ingredient name: reduced-sodium soy sauce
descriptor: reduced-sodium
ingredient without descriptor: soy sauce
preparation: None

1 1/2 tablespoons Asian plum wine
quantity: 1 1/2
measurement: tablespoons
ingredient name: Asian plum wine
descriptor: Asian
ingredient without descriptor: plum wine
preparation: None

1/2 cup white sugar
quantity: 1/2
measurement: cup
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

2 tablespoons ground black pepper
quantity: 2
measurement: tablespoons
ingredient name: ground black pepper
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

1 tablespoon Asian (toasted) sesame oil
quantity: 1
measurement: tablespoon
ingredient name: Asian (toasted) sesame oil
descriptor: Asian
ingredient without descriptor: (toasted) sesame oil
preparation: None

4 pounds Korean-style short ribs (beef chuck flanken, cut 1/3 to 1/2

2 pounds lean ground beef
quantity: 2
measurement: pounds
ingredient name: lean ground beef
descriptor: lean ground
ingredient without descriptor: beef
preparation: None

4 cloves garlic, minced
quantity: 4
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: minced

1 1/2 teaspoons salt
quantity: 1 1/2
measurement: teaspoons
ingredient name: salt
descriptor: None
ingredient without descriptor: salt
preparation: None

1 1/2 teaspoons ground cumin
quantity: 1 1/2
measurement: teaspoons
ingredient name: ground cumin
descriptor: ground
ingredient without descriptor: cumin
preparation: None

1 teaspoon ground turmeric
quantity: 1
measurement: teaspoon
ingredient name: ground turmeric
descriptor: ground
ingredient without descriptor: turmeric
preparation: None

2 tablespoons extra-virgin olive oil
quantity: 2
measurement: tablespoons
ingredient name: extra-virgin olive oil
descriptor: extra-virgin
ingredient without descriptor: oliv

1 (28 ounce) can crushed tomatoes
quantity: 1 (28 ounce)
measurement: can
ingredient name: crushed tomatoes
descriptor: crushed
ingredient without descriptor: tomatoes
preparation: None

2 cloves garlic, crushed
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: crushed

1/2 teaspoon salt
quantity: 1/2
measurement: teaspoon
ingredient name: salt
descriptor: None
ingredient without descriptor: salt
preparation: None

1 1/2 teaspoons dried basil leaves
quantity: 1 1/2
measurement: teaspoons
ingredient name: dried basil leaves
descriptor: dried
ingredient without descriptor: basil leaves
preparation: None

2 (6 ounce) cans tomato paste
quantity: 2 (6 ounce)
measurement: cans
ingredient name: tomato paste
descriptor: None
ingredient without descriptor: tomato paste
preparation: None

1/4 teaspoon ground black pepper
quantity: 1/4
measurement: teaspoon
ingredient name: ground black pepper
descriptor: ground black
ingre

1 teaspoon cayenne pepper
quantity: 1
measurement: teaspoon
ingredient name: cayenne pepper
descriptor: None
ingredient without descriptor: cayenne pepper
preparation: None

1 teaspoon ground black pepper
quantity: 1
measurement: teaspoon
ingredient name: ground black pepper
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

1 tablespoon minced garlic
quantity: 1
measurement: tablespoon
ingredient name: minced garlic
descriptor: minced
ingredient without descriptor: garlic
preparation: None

1 (6 ounce) can tomato paste
quantity: 1 (6 ounce)
measurement: can
ingredient name: tomato paste
descriptor: None
ingredient without descriptor: tomato paste
preparation: None

1 teaspoon paprika
quantity: 1
measurement: teaspoon
ingredient name: paprika
descriptor: None
ingredient without descriptor: paprika
preparation: None

1 red bell pepper, seeded and chopped
quantity: 1
measurement: None
ingredient name: red bell pepper
descriptor: red
ingredient without descr

...

1 tablespoon ketchup
quantity: 1
measurement: tablespoon
ingredient name: ketchup
descriptor: None
ingredient without descriptor: ketchup
preparation: None

1/2 teaspoon paprika
quantity: 1/2
measurement: teaspoon
ingredient name: paprika
descriptor: None
ingredient without descriptor: paprika
preparation: None

Sauce:
https://www.allrecipes.com/recipe/237501/spam-fries-with-spicy-garlic-sriracha-dipping-sauce/
2 (12 ounce) cans fully cooked luncheon meat (such as SPAM®), cut into French fry-size slices
quantity: 2 (12 ounce)
measurement: cans
ingredient name: fully cooked luncheon meat (such as SPAM)
descriptor: fully cooked
ingredient without descriptor: luncheon meat (such as SPAM)
preparation: cut into French fry-size slices

1 cup mayonnaise
quantity: 1
measurement: cup
ingredient name: mayonnaise
descriptor: None
ingredient without descriptor: mayonnaise
preparation: None

1 quart vegetable oil for frying
quantity: 1
measurement: quart
ingredient name: vegetable oil for frying
de

1 (15 ounce) can chili beans in spicy sauce
quantity: 1 (15 ounce)
measurement: can
ingredient name: chili beans in spicy sauce
descriptor: spicy
ingredient without descriptor: chili beans in sauce
preparation: None

1 teaspoon white sugar
quantity: 1
measurement: teaspoon
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

2 (28 ounce) cans diced tomatoes with juice
quantity: 2 (28 ounce)
measurement: cans
ingredient name: diced tomatoes with juice
descriptor: diced
ingredient without descriptor: tomatoes with juice
preparation: None

2 pounds ground beef chuck
quantity: 2
measurement: pounds
ingredient name: ground beef chuck
descriptor: ground
ingredient without descriptor: beef chuck
preparation: None

1 tablespoon dried oregano
quantity: 1
measurement: tablespoon
ingredient name: dried oregano
descriptor: dried
ingredient without descriptor: oregano
preparation: None

1/2 cup beer
quantity: 1/2
measurement: cup
ingredient name: be

3 pounds boneless pork ribs
quantity: 3
measurement: pounds
ingredient name: boneless pork ribs
descriptor: None
ingredient without descriptor: boneless pork ribs
preparation: None

1 (14 ounce) can beef broth
quantity: 1 (14 ounce)
measurement: can
ingredient name: beef broth
descriptor: None
ingredient without descriptor: beef broth
preparation: None

1 (18 ounce) bottle barbeque sauce
quantity: 1 (18 ounce)
measurement: bottle
ingredient name: barbeque sauce
descriptor: barbeque
ingredient without descriptor: sauce
preparation: None

https://www.allrecipes.com/recipe/75861/amazing-pork-tenderloin-in-the-slow-cooker/
1 (1 ounce) envelope dry onion soup mix
quantity: 1 (1 ounce)
measurement: envelope
ingredient name: dry onion soup mix
descriptor: dry
ingredient without descriptor: onion soup mix
preparation: None

freshly ground black pepper to taste
quantity: 
measurement: None
ingredient name: freshly ground black pepper to taste
descriptor: freshly ground black
ingredient without 

2 teaspoons dried parsley
quantity: 2
measurement: teaspoons
ingredient name: dried parsley
descriptor: dried
ingredient without descriptor: parsley
preparation: None

1/2 teaspoon dried thyme
quantity: 1/2
measurement: teaspoon
ingredient name: dried thyme
descriptor: dried
ingredient without descriptor: thyme
preparation: None

1 cup chopped celery
quantity: 1
measurement: cup
ingredient name: chopped celery
descriptor: chopped
ingredient without descriptor: celery
preparation: None

https://www.allrecipes.com/recipe/20096/cheesy-ham-and-hash-brown-casserole/
2 (10.75 ounce) cans condensed cream of potato soup
quantity: 2 (10.75 ounce)
measurement: cans
ingredient name: condensed cream of potato soup
descriptor: condensed
ingredient without descriptor: cream of potato soup
preparation: None

1 (16 ounce) container sour cream
quantity: 1 (16 ounce)
measurement: container
ingredient name: sour cream
descriptor: sour
ingredient without descriptor: cream
preparation: None

8 ounces cooke

5 cups beef broth
quantity: 5
measurement: cups
ingredient name: beef broth
descriptor: None
ingredient without descriptor: beef broth
preparation: None

1 cup thinly sliced carrots
quantity: 1
measurement: cup
ingredient name: thinly sliced carrots
descriptor: thinly sliced
ingredient without descriptor: carrots
preparation: None

1/2 teaspoon dried oregano
quantity: 1/2
measurement: teaspoon
ingredient name: dried oregano
descriptor: dried
ingredient without descriptor: oregano
preparation: None

1 1/2 cups sliced zucchini
quantity: 1 1/2
measurement: cups
ingredient name: sliced zucchini
descriptor: sliced
ingredient without descriptor: zucchini
preparation: None

3 tablespoons chopped fresh parsley
quantity: 3
measurement: tablespoons
ingredient name: chopped fresh parsley
descriptor: chopped fresh
ingredient without descriptor: parsley
preparation: None

https://www.allrecipes.com/recipe/132815/balsamic-roasted-pork-loin/
2 tablespoons steak seasoning rub
quantity: 2
measurement: 

In [555]:
sorted(meat_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True);

In [556]:
meat_ingredients_preparation_dict;

In [557]:
%%time
seafood_ingredients_freq_dict = dict()
seafood_ingredients_measurement_dict = defaultdict(set)
seafood_ingredients_descriptor_dict = defaultdict(set)
seafood_ingredients_preparation_dict = defaultdict(set)

for page in range(2):
    seafood_url = 'https://www.allrecipes.com/recipes/93/seafood/' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(seafood_url)
    for i in ingredients_cat.keys():
        if i in seafood_ingredients_freq_dict:
            seafood_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            seafood_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in seafood_ingredients_measurement_dict:
            seafood_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            seafood_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in seafood_ingredients_descriptor_dict:
            seafood_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            seafood_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in seafood_ingredients_preparation_dict:
            seafood_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            seafood_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/228267/halibut-soft-tacos/
2 tomatoes, chopped
quantity: 2
measurement: None
ingredient name: tomatoes
descriptor: None
ingredient without descriptor: tomatoes
preparation: chopped

1/4 cup chopped red onion
quantity: 1/4
measurement: cup
ingredient name: chopped red onion
descriptor: chopped red
ingredient without descriptor: onion
preparation: None

1 teaspoon olive oil
quantity: 1
measurement: teaspoon
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: None

4 flour tortillas
quantity: 4
measurement: flour
ingredient name: tortillas
descriptor: None
ingredient without descriptor: tortillas
preparation: None

1 teaspoon lemon juice
quantity: 1
measurement: teaspoon
ingredient name: lemon juice
descriptor: None
ingredient without descriptor: lemon juice
preparation: None

4 teaspoons taco sauce
quantity: 4
measurement: teaspoons
ingredient name: taco sauce
descriptor: None
ingredient without descriptor: t

sea salt to taste
quantity: 
measurement: None
ingredient name: sea salt to taste
descriptor: None
ingredient without descriptor: sea salt
preparation: None

2 tablespoons lemon juice
quantity: 2
measurement: tablespoons
ingredient name: lemon juice
descriptor: None
ingredient without descriptor: lemon juice
preparation: None

1 1/2 tablespoons mayonnaise
quantity: 1 1/2
measurement: tablespoons
ingredient name: mayonnaise
descriptor: None
ingredient without descriptor: mayonnaise
preparation: None

1 tablespoon yellow miso paste
quantity: 1
measurement: tablespoon
ingredient name: yellow miso paste
descriptor: yellow
ingredient without descriptor: miso paste
preparation: None

1 pinch cayenne pepper, or to taste
quantity: 1 or to taste
measurement: pinch
ingredient name: cayenne pepper
descriptor: None
ingredient without descriptor: cayenne pepper
preparation: None

2 (8 ounce) center-cut salmon fillets, boned, skin on
quantity: 2 (8 ounce)
measurement: None
ingredient name: center-cu

1/8 teaspoon ground black pepper
quantity: 1/8
measurement: teaspoon
ingredient name: ground black pepper
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

2 tablespoons soy sauce
quantity: 2
measurement: tablespoons
ingredient name: soy sauce
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

1 pound salmon
quantity: 1
measurement: pound
ingredient name: salmon
descriptor: None
ingredient without descriptor: salmon
preparation: None

1/4 teaspoon garlic salt
quantity: 1/4
measurement: teaspoon
ingredient name: garlic salt
descriptor: None
ingredient without descriptor: garlic salt
preparation: None

1/4 cup maple syrup
quantity: 1/4
measurement: cup
ingredient name: maple syrup
descriptor: None
ingredient without descriptor: maple syrup
preparation: None

1 clove garlic, minced
quantity: 1
measurement: None
ingredient name: clove garlic
descriptor: None
ingredient without descriptor: clove garlic
preparation: minced

https://ww

2 teaspoons dried oregano
quantity: 2
measurement: teaspoons
ingredient name: dried oregano
descriptor: dried
ingredient without descriptor: oregano
preparation: None

2 teaspoons Cajun seasoning
quantity: 2
measurement: teaspoons
ingredient name: Cajun seasoning
descriptor: None
ingredient without descriptor: Cajun seasoning
preparation: None

1 large green bell pepper, chopped
quantity: 1
measurement: None
ingredient name: large green bell pepper
descriptor: large green
ingredient without descriptor: bell pepper
preparation: chopped

1 teaspoon cayenne pepper
quantity: 1
measurement: teaspoon
ingredient name: cayenne pepper
descriptor: None
ingredient without descriptor: cayenne pepper
preparation: None

1 large onion, chopped
quantity: 1
measurement: None
ingredient name: large onion
descriptor: large
ingredient without descriptor: onion
preparation: chopped

1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes
quantity: 1
measurement: pound
ingredient name: skin

2 (10.75 ounce) cans condensed cream of mushroom soup
quantity: 2 (10.75 ounce)
measurement: cans
ingredient name: condensed cream of mushroom soup
descriptor: condensed
ingredient without descriptor: cream of mushroom soup
preparation: None

1 (12 ounce) package egg noodles
quantity: 1 (12 ounce)
measurement: package
ingredient name: egg noodles
descriptor: None
ingredient without descriptor: egg noodles
preparation: None

2 cups shredded Cheddar cheese
quantity: 2
measurement: cups
ingredient name: shredded Cheddar cheese
descriptor: shredded
ingredient without descriptor: Cheddar cheese
preparation: None

2 (5 ounce) cans tuna, drained
quantity: 2 (5 ounce)
measurement: cans
ingredient name: tuna
descriptor: None
ingredient without descriptor: tuna
preparation: drained

1/2 (4.5 ounce) can sliced mushrooms
quantity: 1/2 (4.5 ounce)
measurement: can
ingredient name: sliced mushrooms
descriptor: sliced
ingredient without descriptor: mushrooms
preparation: None

1 cup frozen green peas

...

2 tablespoons fine bread crumbs
quantity: 2
measurement: tablespoons
ingredient name: fine bread crumbs
descriptor: fine
ingredient without descriptor: bread crumbs
preparation: None

1 (10 ounce) package frozen chopped spinach, thawed and drained
quantity: 1 (10 ounce)
measurement: package
ingredient name: frozen chopped spinach
descriptor: frozen chopped
ingredient without descriptor: spinach
preparation: thawed and drained

1/2 cup milk
quantity: 1/2
measurement: cup
ingredient name: milk
descriptor: None
ingredient without descriptor: milk
preparation: None

seasoned salt to taste
quantity: 
measurement: None
ingredient name: seasoned salt to taste
descriptor: seasoned
ingredient without descriptor: salt
preparation: None

8 ounces Monterey Jack cheese, shredded
quantity: 8
measurement: ounces
ingredient name: Monterey Jack cheese
descriptor: None
ingredient without descriptor: Monterey Jack cheese
preparation: shredded

https://www.allrecipes.com/recipe/17753/coconut-shrimp-i/
1 1

1/4 cup finely chopped pecans
quantity: 1/4
measurement: cup
ingredient name: finely chopped pecans
descriptor: finely chopped
ingredient without descriptor: pecans
preparation: None

salt and pepper to taste
quantity: 
measurement: None
ingredient name: salt and pepper to taste
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1/4 cup dry bread crumbs
quantity: 1/4
measurement: cup
ingredient name: dry bread crumbs
descriptor: dry
ingredient without descriptor: bread crumbs
preparation: None

1/4 cup butter, melted
quantity: 1/4
measurement: cup
ingredient name: butter
descriptor: None
ingredient without descriptor: butter
preparation: melted

1 1/2 tablespoons honey
quantity: 1 1/2
measurement: tablespoons
ingredient name: honey
descriptor: None
ingredient without descriptor: honey
preparation: None

3 tablespoons Dijon mustard
quantity: 3
measurement: tablespoons
ingredient name: Dijon mustard
descriptor: None
ingredient without descriptor: Dijon mus

2 tablespoons cornstarch
quantity: 2
measurement: tablespoons
ingredient name: cornstarch
descriptor: None
ingredient without descriptor: cornstarch
preparation: None

1 teaspoon ground cayenne pepper
quantity: 1
measurement: teaspoon
ingredient name: ground cayenne pepper
descriptor: ground
ingredient without descriptor: cayenne pepper
preparation: None

1 cup beer
quantity: 1
measurement: cup
ingredient name: beer
descriptor: None
ingredient without descriptor: beer
preparation: None

1 egg
quantity: 1
measurement: None
ingredient name: egg
descriptor: None
ingredient without descriptor: egg
preparation: None

1/2 teaspoon dried dill weed
quantity: 1/2
measurement: teaspoon
ingredient name: dried dill weed
descriptor: dried
ingredient without descriptor: dill weed
preparation: None

1 teaspoon baking powder
quantity: 1
measurement: teaspoon
ingredient name: baking powder
descriptor: None
ingredient without descriptor: baking powder
preparation: None

1/2 teaspoon salt
quantity: 1/2
m

1 pound medium shrimp, peeled and deveined
quantity: 1
measurement: pound
ingredient name: medium shrimp
descriptor: medium
ingredient without descriptor: shrimp
preparation: peeled and deveined

1/4 cup chopped parsley
quantity: 1/4
measurement: cup
ingredient name: chopped parsley
descriptor: chopped
ingredient without descriptor: parsley
preparation: None

1 (15 ounce) jar Alfredo sauce
quantity: 1 (15 ounce)
measurement: jar
ingredient name: Alfredo sauce
descriptor: None
ingredient without descriptor: Alfredo sauce
preparation: None

2 tablespoons extra-virgin olive oil
quantity: 2
measurement: tablespoons
ingredient name: extra-virgin olive oil
descriptor: extra-virgin
ingredient without descriptor: olive oil
preparation: None

1/4 cup butter
quantity: 1/4
measurement: cup
ingredient name: butter
descriptor: None
ingredient without descriptor: butter
preparation: None

1 red bell pepper, diced
quantity: 1
measurement: None
ingredient name: red bell pepper
descriptor: red
ingredie

1/2 teaspoon lemon zest
quantity: 1/2
measurement: teaspoon
ingredient name: lemon zest
descriptor: None
ingredient without descriptor: lemon zest
preparation: None

https://www.allrecipes.com/recipe/18871/easy-tuna-casserole/
1 (5 ounce) can tuna, drained
quantity: 1 (5 ounce)
measurement: can
ingredient name: tuna
descriptor: None
ingredient without descriptor: tuna
preparation: drained

3 cups cooked macaroni
quantity: 3
measurement: cups
ingredient name: cooked macaroni
descriptor: cooked
ingredient without descriptor: macaroni
preparation: None

1 cup shredded Cheddar cheese
quantity: 1
measurement: cup
ingredient name: shredded Cheddar cheese
descriptor: shredded
ingredient without descriptor: Cheddar cheese
preparation: None

1 1/2 cups French fried onions
quantity: 1 1/2
measurement: cups
ingredient name: French fried onions
descriptor: French fried
ingredient without descriptor: onions
preparation: None

1 (10.75 ounce) can condensed cream of chicken soup
quantity: 1 (10.75 ou

In [558]:
sorted(seafood_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True);

In [559]:
seafood_ingredients_preparation_dict;

In [560]:
ingredients_categorized = {}
ingredients_categorized['vegetarian_protein'] = {}
ingredients_categorized['vegetarian_protein']['ingredients'] = vegetarian_protein_ingredients_freq_dict
ingredients_categorized['vegetarian_protein']['measurement'] = vegetarian_protein_ingredients_measurement_dict
ingredients_categorized['vegetarian_protein']['descriptor'] = vegetarian_protein_ingredients_descriptor_dict
ingredients_categorized['vegetarian_protein']['preparation'] = vegetarian_protein_ingredients_preparation_dict
ingredients_categorized['meat'] = {}
ingredients_categorized['meat']['ingredients'] = meat_ingredients_freq_dict
ingredients_categorized['meat']['measurement'] = meat_ingredients_measurement_dict
ingredients_categorized['meat']['descriptor'] = meat_ingredients_descriptor_dict
ingredients_categorized['meat']['preparation'] = meat_ingredients_preparation_dict
ingredients_categorized['seafood'] = {}
ingredients_categorized['seafood']['ingredients'] = seafood_ingredients_freq_dict
ingredients_categorized['seafood']['measurement'] = seafood_ingredients_measurement_dict
ingredients_categorized['seafood']['descriptor'] = seafood_ingredients_descriptor_dict
ingredients_categorized['seafood']['preparation'] = seafood_ingredients_preparation_dict

In [561]:
# save scraped data
with open('ingredients_categorized.pickle', 'wb') as file:
    pickle.dump(ingredients_categorized, file, protocol=pickle.HIGHEST_PROTOCOL)
file.closed

True

In [562]:
# check
with open('ingredients_categorized.pickle', 'rb') as file:
    ingredients_categorized = pickle.load(file)
            
ingredients_categorized['meat']['descriptor'].keys()

dict_keys(['Cheddar cheese', 'water', 'beef', 'tomato soup', 'water as', 'rice', 'Worcestershire sauce', 'onion', 'bell peppers', 'salt and pepper', 'tomatoes', 'salt', 'chili powder', 'soy sauce', 'oregano', 'pepper sauce (such as Tabasco)', 'bell pepper', 'elbow macaroni', 'pepper', 'tomato sauce', 'basil', 'beef broth', 'garlic powder', 'rolls', 'leaf', 'garlic', 'chopped giardiniera (pickled Italian vegetables)', 'boneless beef chuck', 'pepper flakes', 'dried rosemary', 'parsley', 'vinegar', 'chicken broth', 'oil', 'thyme', 'Korean-style short ribs (beef chuck flanken)', 'plum wine', 'sugar', 'honey', '(toasted) sesame oil', 'onion powder', 'cumin', 'olives', 'olive oil', 'sauce (such as Louisiana)', 'sazon seasoning (such as Badia )', 'bay leaf', 'small butternut squash', 'cooking wine', 'milk', 'ground veal', 'cooking oats', 'eggs', 'cooking spray', 'ketchup', 'pork', 'cider vinegar', 'mustard', 'turmeric', 'cilantro', 'water (optional)', 'ginger root', 'peas', 'potatoes', 'cayen